# Land

# <font color="red">NOTE: NOTEBOOK 4 IS NOT FINISHED YET</font>

In [ ]:
# Change these as desired

# The slug of the locality you are currently working on
locality = "us-nc-guilford"

# Whether to print out a lot of stuff (can help with debugging) or stay mostly quiet
verbose = True

# Clear previous state for this notebook and start fresh
clear_checkpoints = False

# Clear cached results for model predictions and start fresh
clear_model_results = False

# 1. Basic Setup

In [ ]:
import init_notebooks
init_notebooks.setup_environment()

In [ ]:
# import a bunch of stuff
from openavmkit.pipeline import (
    init_notebook,
    read_pickle,
    load_settings
)

In [ ]:
init_notebook(locality)

In [ ]:
if clear_checkpoints:
    delete_checkpoints("4-land")

# 2. Run Land Analysis

In [ ]:
# load the data
sales_univ_pair = read_pickle("out/2-clean-sup")

In [ ]:
from openavmkit.land import run_land_analysis

run_land_analysis(sales_univ_pair, load_settings(), verbose)

In [ ]:
import pandas as pd

path = "out/models/single_family/vacant/ensemble/pred_universe.parquet"
df = pd.read_parquet(path)
df = df[["key","prediction","assr_land_value","land_area_sqft"]]
df = df.merge(sales_univ_pair.universe[["key","latitude","longitude"]], on="key", how="left")

df = df.rename(columns={"prediction":"land_value"})
df = df[df["land_value"].gt(0) & df["land_area_sqft"].gt(0)]
df

In [ ]:
from openavmkit.land import fit_land_SLICE_model, LandSLICEModel

slice_model = fit_land_SLICE_model(df, size_field="land_area_sqft", value_field="land_value", verbose=True)
slice_model.predict(df)

df

In [ ]:
from openavmkit.utilities.plotting import plot_scatterplot

# def plot_scatterplot(
#     df,
#     x: str,
#     y: str,
#     xlabel: str = None,
#     ylabel: str = None,
#     title: str = None,
#     out_file: str = None,
#     style: dict = None,
#     best_fit_line: bool = False,
#     perfect_fit_line: bool = False,
#     metadata_field: str = None,
# ):

df["land_area_acres"] = df["land_area_sqft"]/43560

plot_scatterplot(df, "land_area_acres", "size_factor", "Land size", "Size factor", "Land size -> Size factor")


#df[["size_factor", "land_area_sqft"]]

In [ ]:
df_univ = sales_univ_pair.universe.copy()
df_univ = df_univ.merge(df[["key","location_factor","size_factor","land_value"]], on="key", how="left")

df_univ.to_parquet("land_test.parquet")